This script contains the data preparation of Hong Kong monthly Ovitrap Index in different locations. The data is from Food and Environmental Hygiene Department(FEHD). The data from 2008 to 2017 is downloaded from FEHD in pdf format while the data from 2018 to now (July, 2018) is scrapped from the FEHD website. The processed data is converted to a csv file at the end of this script and it will be stored into a relational database in the next step.

In [1]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup

def matched(patterns,values): #find out expressions that matched with the patterns
    found = []
    for pattern in patterns:
        for value in values:
            match = re.search(pattern,value)
            if match:
                found.append(value)
    return set(found)

def replace_value(to_replace,replace,df_column): #replace value in the dataframe column
    for index,value in enumerate(to_replace): 
        df_column = df_column.replace(value,replace[index])
    return df_column

def make_row(row_list): 
    months = ['{:02d}'.format(i) for i in range(1,13)]
    row_dict = {}
    for index,value in enumerate(months):
        row_dict[value] = { 'Eng': row_list[0],
                            'Chi': row_list[1],
                            'Date': "{month}-{year}".format(month=value,year=row_list[-1]),
                            'AOI': row_list[index+2]
                            }   
    return row_dict

def convert_float(number):
    if '/' in number:
        decimal = float(number[0:-2])/100
        return round(decimal,3)
    elif number[-1] == '%':
        decimal = float(number[0:-1])/100
        return round(decimal,3)
    else:
        decimal = float(number)/100
        return round(decimal,3)    

def classification(value):
    if value < 0.05:
        return 1
    elif value >= 0.05 and value < 0.2:
        return 2
    elif value >= 0.2 and value < 0.4:
        return 3
    elif value >= 0.4:
        return 4

In [8]:
archive = pd.read_csv("monthlyOvitrap_2008-2017.csv",header=None)

#use the table header at the fourth row as the header of the dataframe
#only need the English parts
new_col = {}
for i in range(13):
    splited = archive.loc[4,i].split('\n')
    new_col[i] = splited[0]

archive.rename(columns=new_col,inplace=True)

#find and drop all the rows that contain unwanted data
patterns = ["Food and Environmental Hygiene Department","食物環境衛生署","^Monthly",
            "二零..年每月誘蚊產卵器分區指數","Locations\n地區"]

unwanted = matched(patterns,archive["Locations"])
for value in unwanted:
    drop_index = archive[archive["Locations"] == value].index
    archive.drop(drop_index,inplace=True)

archive.reset_index(drop=True,inplace=True)    

#consistency is provided for inconsistent names of locations 
to_replace_1 = ['Central, Sheung Wan and Sai Ying\nPun\n中環, 上環及西營盤','Tseung Kwan O\n將軍澳',
                'Tseung Kwan O South (Formerly: Tseung Kwan O)\n將軍澳南 (前稱: 將軍澳)',
                'Tsing Yi\n青衣','Tsing Yi South (Formerly: Tsing Yi)\n青衣南 (前稱: 青衣)']

replace_1 = ['Central, Sheung Wan and Sai Ying Pun\n中環, 上環及西營盤','Tseung Kwan O South\n將軍澳南',
             'Tseung Kwan O South\n將軍澳南','Tsing Yi South\n青衣南','Tsing Yi South\n青衣南']

archive['Locations'] = replace_value(to_replace_1,replace_1,archive['Locations'])

#Columns 'Eng' and 'Chi' are created to store the bilingual location 
for index, value in enumerate(archive['Locations']):
    splited = value.split('\n')
    archive.loc[index,'Eng'] = splited[0]
    archive.loc[index,'Chi'] = splited[1]

archive.drop('Locations',axis=1,inplace=True)
cols = archive.columns.tolist()
cols = cols[-2:]+cols[:-2]
archive = archive[cols]

#Column 'Year' is created to stored the corresponding year
archive.loc[:,'Year'] = pd.Series('NaN'for each in range(archive.shape[0]))
year_interval = archive[archive['Eng'] == 'Chai Wan West'].index.tolist()
for index, value in enumerate(range(2008,2018)):
    try:
        archive['Year'].loc[year_interval[index]:year_interval[index+1]] = value
    except IndexError:
        archive['Year'].loc[year_interval[index]:] = value

In [7]:
response = requests.get("https://www.fehd.gov.hk/tc_chi/pestcontrol/dengue_fever/ovitrap_index.html")
content = response.content
parser =  BeautifulSoup(content,'html.parser')

scrapped = []
for each in parser.select("td"):
    scrapped.append(each.text)
    
scrapped = scrapped[10:] # The first ten data are unrelated
pattern = ['區$','月$'] # Data that match with anyone of these two Chinese words is not needed
unwanted = matched(pattern,scrapped)

wanted = []
for value in scrapped:
    if value not in unwanted:
        wanted.append(value)

interval = np.arange(0,len(wanted),13) # Every row is consisted of 13 data
current = []
for index,value in enumerate(interval):
    location = wanted[value]
    if location == "香港國際機場": # Port area are not the targets in this analysis
        break
    else:
        current.append(wanted[value:interval[index+1]])

for each in current: 
    each.insert(0,'') # Prepare a place for the 'Eng' column 
    each.append(2018) # Provide value for the 'Year' column
    

header = archive.columns.tolist() # set the same header for combination of datasets
current = pd.DataFrame(current,columns=header)
current.replace('',np.nan,inplace=True)     

In [9]:
combined = pd.concat([archive,current],axis=0,ignore_index=True)

to_replace_2 = ['中環, 上環及西營盤','上環及西營盤(前稱: 中環,上環及西營盤)','將軍澳南(前稱: 將軍澳)','青衣南(前稱: 青衣)']
replace_2 = ['上環及西營盤','上環及西營盤','將軍澳南','青衣南']
combined['Chi'] = replace_value(to_replace_2,replace_2,combined['Chi'])

to_replace_3 = ['Central, Sheung Wan and Sai Ying Pun','Tuen Mun (S)','Tuen Mun (N)']
replace_3 = ['Sheung Wan and Sai Ying Pun','Tuen Mun South','Tuen Mun North'] 
combined['Eng'] = replace_value(to_replace_3,replace_3,combined['Eng'])

eng_name = combined['Eng'].unique()
chi_name = combined['Chi'].unique()
eng_name = [name for name in eng_name if name is not np.nan]
translations = list(zip(eng_name,chi_name))

new_locations = [('Central and Admiralty','中環及金鐘'),('Yau Tong','油塘'),('Wo Che','禾輋'),
                 ('Tuen Mun West','屯門西'),('Tsuen Wan West','荃灣西')]
translations = translations + new_locations

name_2018 = combined[combined['Year']==2018].loc[:,['Eng','Chi']]
for translation in translations:
    for index in name_2018.index:
        if name_2018.loc[index][1] == translation[1]:
            name_2018.loc[index][0] = translation[0]  
combined.loc[name_2018.index,'Eng'] = name_2018['Eng']         

In [22]:
complete = pd.DataFrame(columns=['Eng','Chi','Date','AOI'])

for i in combined.index:
    row_list = combined.loc[i].tolist()
    row_dict = make_row(row_list)
    for key, value in row_dict.items():
        complete = complete.append(value,ignore_index=True)

complete.dropna(inplace=True)
complete.reset_index(drop=True,inplace=True)
complete['AOI'] = complete['AOI'].apply(convert_float)
complete['Classification'] = complete['AOI'].apply(classification)

In [6]:
complete.to_csv('Area_OviTrap_Index_Jan2008-Jul2018.csv',index=False)

In [20]:
combined

,Eng,Chi,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,Year
0,Chai Wan West,柴灣西,0.0%,0.0%,0.0%,5.7%,7.3%,23.6%,7.8%,9.6%,5.4%,1.9%,0.0%,0.0%,2008
1,Wan Chai North,灣仔北,0.0%,0.0%,0.0%,0.0%,0.0%,6.0%,3.8%,3.8%,1.9%,3.7%,1.9%,0.0%,2008
2,Happy Valley,跑馬地,0.0%,0.0%,0.0%,0.0%,1.8%,18.2%,9.3%,7.4%,7.3%,12.0%,1.8%,0.0%,2008
3,Sheung Wan and Sai Ying Pun,上環及西營盤,0.0%,0.0%,0.0%,1.9%,7.9%,10.9%,7.4%,5.7%,13.0%,9.4%,3.8%,0.0%,2008
4,Sai Wan,西環,0.0%,0.0%,0.0%,0.0%,9.3%,5.9%,3.8%,13.0%,13.2%,3.8%,5.7%,0.0%,2008
5,North Point,北角,0.0%,0.0%,0.0%,0.0%,0.0%,15.1%,18.9%,11.8%,6.1%,0.0%,0.0%,0.0%,2008
6,Aberdeen and Ap Lei Chau,香港仔及鴨脷洲,0.0%,0.0%,0.0%,5.5%,7.3%,15.4%,11.5%,7.7%,7.8%,1.9%,1.9%,0.0%,2008
7,Pokfulam,薄扶林,0.0%,0.0%,0.0%,3.6%,9.1%,3.6%,16.4%,10.9%,7.4%,3.6%,0.0%,0.0%,2008
8,Cheung Chau,長洲,0.0%,0.0%,0.0%,2.8%,11.1%,8.3%,8.3%,2.9%,5.7%,2.9%,2.9%,0.0%,2008
9,Tung Chung,東涌,0.0%,0.0%,0.0%,0.0%,11.1%,2.9%,11.1%,8.6%,29.4%,5.7%,0.0%,0.0%,2008


In [23]:
complete

,Eng,Chi,Date,AOI,Classification
0,Chai Wan West,柴灣西,01-2008,0.000,1
1,Chai Wan West,柴灣西,02-2008,0.000,1
2,Chai Wan West,柴灣西,03-2008,0.000,1
3,Chai Wan West,柴灣西,04-2008,0.057,2
4,Chai Wan West,柴灣西,05-2008,0.073,2
5,Chai Wan West,柴灣西,06-2008,0.236,3
6,Chai Wan West,柴灣西,07-2008,0.078,2
7,Chai Wan West,柴灣西,08-2008,0.096,2
8,Chai Wan West,柴灣西,09-2008,0.054,2
9,Chai Wan West,柴灣西,10-2008,0.019,1
